# Collect results
Here we perform MRI/US fusion using the trained VoxelMorph models and collect the results

In [1]:
from notebooks.setup import test_generator, model, config, latest_checkpoint
from notebooks.utils import dice_coeff
import itertools
import numpy as np

Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


vxm info: mutual information loss is experimental


Prepare storing function

In [2]:
from tqdm.notebook import tqdm
from skimage import measure

def create_and_store_results(model, generator, filename):
    results = {
        "prostate_dice": [], "target_dice": [], "target_error": [],
        "prostate_dice_def": [], "target_dice_def": [], "target_error_def": [],
        "prostate_dice_pre": [], "target_dice_pre": [], "target_error_pre": []
    }
    for idx in tqdm(range(len(generator))):
        test_input, test_output, mr_targets, us_targets, _ = generator(idx)

        test_pred = model.predict(test_input)
        prostate_dice_val = dice_coeff(test_pred[2], test_output[2])
        results["prostate_dice"].append(prostate_dice_val)
        prostate_dice_val = dice_coeff(test_pred[2], test_input[2])
        results["prostate_dice_def"].append(prostate_dice_val)
        prostate_dice_val = dice_coeff(test_input[2], test_output[2])
        results["prostate_dice_pre"].append(prostate_dice_val)

        for i, (mr_target, us_target) in enumerate(itertools.zip_longest(mr_targets, us_targets)):
            if mr_target is None or us_target is None:
                print(f" /!\  test data [{idx}] has unpaired targets [{i}]")
                continue

            for select_target in range(min(mr_target.shape[-1], us_target.shape[-1])):
                test_input_target = mr_target[np.newaxis, ..., [select_target]]
                test_output_target = us_target[np.newaxis, ..., [select_target]]
                test_pred_target = model.apply_transform(test_input[0], test_input[1], test_input_target)

                # in "error" we use  *0.5 because images have an isotropic spacing of 0.5mm

                # Dice between target in MR_def and US
                target_dice_val = dice_coeff(test_pred_target, test_output_target)
                target_error_val = np.linalg.norm((measure.centroid(test_pred_target) - measure.centroid(test_output_target))) * 0.5
                results["target_dice"].append(target_dice_val)
                results["target_error"].append(target_error_val)

                # Dice between target in MR and MR_def
                target_dice_val = dice_coeff(test_pred_target, test_input_target)
                target_error_val = np.linalg.norm((measure.centroid(test_pred_target) - measure.centroid(test_input_target))) * 0.5
                results["target_dice_def"].append(target_dice_val)
                results["target_error_def"].append(target_error_val)

                # Dice between target in MR and US
                target_dice_val = dice_coeff(test_input_target, test_output_target)
                target_error_val = np.linalg.norm((measure.centroid(test_input_target) - measure.centroid(test_output_target))) * 0.5
                results["target_dice_pre"].append(target_dice_val)
                results["target_error_pre"].append(target_error_val)

    # convert everything to ndarray
    for k in results.keys():
        results[k] = np.array(results[k])

    # store results
    try:
        np.savez_compressed(filename, results)
    except Exception:
        print("Error saving the results!")

    return results

### Run the evaluation

Run for a single model

In [3]:
# instantiate the network and load the model
model.load_weights(latest_checkpoint)

filename = f"../../results/results_{config['name']}"
create_and_store_results(model, test_generator, filename)

  0%|          | 0/96 [00:00<?, ?it/s]

 /!\  test data [48] has unpaired targets [0]
 /!\  test data [66] has unpaired targets [0]


{'prostate_dice': array([0.93233704, 0.80693526, 0.91856753, 0.89926599, 0.87984138,
        0.90226867, 0.88160838, 0.90901067, 0.92525892, 0.89105792,
        0.89864985, 0.92660821, 0.84010065, 0.93068843, 0.91991185,
        0.90918233, 0.87873114, 0.78478112, 0.912643  , 0.75700415,
        0.90528578, 0.91951357, 0.87447211, 0.8923057 , 0.90296393,
        0.92210983, 0.89915367, 0.90730195, 0.90505327, 0.90043058,
        0.92422884, 0.93001554, 0.92701784, 0.89742262, 0.88503273,
        0.89373128, 0.85093318, 0.93154121, 0.92629778, 0.89416135,
        0.90432243, 0.93668827, 0.87348568, 0.89081687, 0.93974788,
        0.90639569, 0.9119459 , 0.93767601, 0.91957333, 0.93214844,
        0.93093394, 0.91192394, 0.91999976, 0.90229611, 0.92027202,
        0.91221205, 0.91368741, 0.90133231, 0.90410086, 0.92363377,
        0.92497406, 0.92778322, 0.90280658, 0.92306427, 0.88013928,
        0.91738698, 0.86559373, 0.84006449, 0.88235396, 0.899946  ,
        0.92738266, 0.88686321,

Run for all the models

In [ ]:
import tensorflow as tf
from pathlib import Path
from notebooks.setup import size
from notebooks.utils import prepare_model

model = prepare_model(inshape=size, sim_param=0, lambda_param=0, gamma_param=0)
for checkpoints_dir in Path(f"../../models").iterdir():
    info = checkpoints_dir.name.split("_")
    if len(info) > 3:
        print("special model, ignoring")
        continue
    _, lambda_param, gamma_param = info
    latest_checkpoint = tf.train.latest_checkpoint(checkpoints_dir / "checkpoints")
    model.load_weights(latest_checkpoint)

    filename = f"../../results/results_{lambda_param}_{gamma_param}"
    create_and_store_results(model, test_generator, filename)